# Project Title
## Tourism Experience Analytics Using Regression, Classification and Recommendation System

## Github Link for this project :- https://github.com/shabbu8111999/Tourism_Experience_Analytics

## Problem Statement
### Tourism platforms generate large volumes of data related to user visits, attraction details, travel modes, and ratings. However, this data is often underutilized and stored across multiple datasets, making it difficult to extract meaningful insights and provide personalized experiences to users. The objective of this project is to analyze tourism-related data by integrating multiple datasets into a single consolidated dataset and applying machine learning techniques to:

### - Predict the rating a user is likely to give to a tourist attraction.

### - Classify the user’s visit mode (such as Business, Family, Couples, or Friends) based on historical travel behavior and demographic information.

### - Recommend tourist attractions to users based on their preferences and past interactions.

### By leveraging data cleaning, exploratory data analysis, and machine learning models, this project aims to help tourism businesses improve customer satisfaction, enable data-driven decision-making, and deliver personalized attraction recommendations.

## Importing Necessary Libraries

In [1]:
# For System Operations
import os
import sys

# For Data Manipulation
import numpy as np
import pandas as pd

# For Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
sns.set(style="whitegrid")
plt.style.use("seaborn-v0_8")

# Statistical Analysis and Hypothesis Testing
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    StandardScaler
)

# Model Evaluation
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# Utilities
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Moving one level up to the project root folder
os.path.abspath('../')

'c:\\Users\\H P\\Desktop\\Tourism_Experience_Analytics'

## Dataset Loading and Initial Inspection

### Loading all the Datasets

In [4]:
# Loading Data
user_df = pd.read_excel('../data/User.xlsx')
transaction_df = pd.read_excel('../data/Transaction.xlsx')
type_df = pd.read_excel('../data/Type.xlsx')
region_df = pd.read_excel('../data/Region.xlsx')
mode_df = pd.read_excel('../data/Mode.xlsx')
item_df = pd.read_excel('../data/Item.xlsx')
country_df = pd.read_excel('../data/Country.xlsx')
continent_df = pd.read_excel('../data/Continent.xlsx')
city_df = pd.read_excel('../data/City.xlsx')

print("Datasets Loaded Successfully")

Datasets Loaded Successfully


#### Datasets have loaded successfully, I used openpyxl package to load the .xlsx files.

### Structural Inspection

In [5]:
# Dataset's Shapes

datasets = {
    "User": user_df,
    "Transaction": transaction_df,
    "Type": type_df,
    "Region": region_df,
    "Mode": mode_df,
    "Item": item_df,
    "Country": country_df,
    "Continent": continent_df,
    "City": city_df
}

for name, df in datasets.items():
    print(f"{name} Dataset Shape: {df.shape}")

User Dataset Shape: (33530, 5)
Transaction Dataset Shape: (52930, 7)
Type Dataset Shape: (17, 2)
Region Dataset Shape: (22, 3)
Mode Dataset Shape: (6, 2)
Item Dataset Shape: (30, 5)
Country Dataset Shape: (165, 3)
Continent Dataset Shape: (6, 2)
City Dataset Shape: (9143, 3)


#### There are total of 9 datasets and each datasets have different rows and columns, some datasets have less number of rows and columns compared to the most numbers like User Data and Transaction Data.

### Column Level Inspection

In [6]:
# Column wise overview of each dataset
for name, df in datasets.items():
    print(f"\n{name} Dataset Columns:")
    print(df.columns.tolist())


User Dataset Columns:
['UserId', 'ContinentId', 'RegionId', 'CountryId', 'CityId']

Transaction Dataset Columns:
['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode', 'AttractionId', 'Rating']

Type Dataset Columns:
['AttractionTypeId', 'AttractionType']

Region Dataset Columns:
['Region', 'RegionId', 'ContinentId']

Mode Dataset Columns:
['VisitModeId', 'VisitMode']

Item Dataset Columns:
['AttractionId', 'AttractionCityId', 'AttractionTypeId', 'Attraction', 'AttractionAddress']

Country Dataset Columns:
['CountryId', 'Country', 'RegionId']

Continent Dataset Columns:
['ContinentId', 'Continent']

City Dataset Columns:
['CityId', 'CityName', 'CountryId']


#### These are the Column names of each and every datasets, I used Loop to simply show the dataset names in a list format easy to visualize.

### Sample Rows

In [7]:
# First few rows of each dataset
for name, df in datasets.items():
    print(f"\n{name} Dataset Preview:")
    display(df.head())


User Dataset Preview:


,UserId,ContinentId,RegionId,CountryId,CityId
0,14,5,20,155,220.0
1,16,3,14,101,3098.0
2,20,4,15,109,4303.0
3,23,1,4,22,154.0
4,25,3,14,101,3098.0



Transaction Dataset Preview:


,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating
0,3,70456,2022,10,2,640,5
1,8,7567,2022,10,4,640,5
2,9,79069,2022,10,3,640,5
3,10,31019,2022,10,3,640,3
4,15,43611,2022,10,2,640,3



Type Dataset Preview:


...

,CountryId,Country,RegionId
0,0,-,0
1,1,Cameroon,1
2,2,Chad,1
3,3,Rwanda,1
4,4,Ethiopia,2



Continent Dataset Preview:


,ContinentId,Continent
0,0,-
1,1,Africa
2,2,America
3,3,Asia
4,4,Australia & Oceania



City Dataset Preview:


,CityId,CityName,CountryId
0,0,-,0
1,1,Douala,1
2,2,South Region,1
3,3,N'Djamena,2
4,4,Kigali,3


#### These are the First Five rows of every datasets including every column names and information.

In [8]:
# Last few rows of each dataset
for name, df in datasets.items():
    print(f"\n{name} Dataset Last 5 Rows:")
    display(df.tail())


User Dataset Last 5 Rows:


,UserId,ContinentId,RegionId,CountryId,CityId
33525,88179,5,21,162,7833.0
33526,88185,3,12,80,2534.0
33527,88187,3,12,88,2604.0
33528,88189,5,17,131,6129.0
33529,88190,5,21,159,7494.0



Transaction Dataset Last 5 Rows:


,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating
52925,211227,87100,2018,9,2,1297,4
52926,211238,88112,2016,2,2,1297,5
52927,211239,88112,2016,2,2,1297,4
52928,211240,88112,2016,2,2,1297,4
52929,211241,88112,2016,2,2,1297,5



Type Dataset Last 5 Rows:


...

,CountryId,Country,RegionId
160,160,Ireland,21
161,161,Monaco,21
162,162,Netherlands,21
163,163,United Kingdom,21
164,164,Yemen,12



Continent Dataset Last 5 Rows:


,ContinentId,Continent
1,1,Africa
2,2,America
3,3,Asia
4,4,Australia & Oceania
5,5,Europe



City Dataset Last 5 Rows:


,CityId,CityName,CountryId
9138,9138,Yeovil,163
9139,9139,York,163
9140,9140,Yorkshire,163
9141,9141,Zaandam,163
9142,9142,Sanaa,164


#### These are the last five rows of every datasets, including every column names and it's information.

## Merging the Datasets

### Merging Transactions and User

In [10]:
# Transaction and User Dataset Merge
merged_df = transaction_df.merge(
    user_df,
    on = "UserId",
    how = "left"
)

print("After Merging the Transaction and User Dataset:", merged_df.shape)

After Merging the Transaction and User Dataset: (52930, 11)


#### After Merging the Transaction and User data based on Left Join, the shapes are now set into 52,930 Rows and 11 Columns.

### Merging City Data

In [11]:
# Merging the City Dataset
merged_df = merged_df.merge(
    city_df,
    left_on = "CityId",
    right_on = "CityId",
    how = "left"
)

print("After Merging the City Dataset:", merged_df.shape)

After Merging the City Dataset: (52930, 13)


#### After merging the City data Features has increased by 2, now the shapes are 52,930 rows and 13 columns.

### Merging Country Data

In [ ]:
# Checking the Merged Data Column names
merged_df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode',
       'AttractionId', 'Rating', 'ContinentId', 'RegionId', 'CountryId_x',
       'CityId', 'CityName', 'CountryId_y'],
      dtype='str')

In [ ]:
# Dropping CountryId_y and Renaming CountryId_x to CountryId
merged_df.drop(columns=["CountryId_y"], inplace=True)
merged_df.rename(columns={"CountryId_x": "CountryId"}, inplace=True)

#### Dropping the CountryId_y, because there are 2 CountryId which was merged earlier, so one was duplicate I droped it and then Renamed CountryId_x into CountryId.

In [18]:
# Merging the Country Dataset
merged_df = merged_df.merge(
    country_df,
    on = "CountryId",
    how = "left"
)

print("After Merging the Country Dataset:", merged_df.shape)

After Merging the Country Dataset: (52930, 14)


### Merging the Region

In [21]:
merged_df.drop(columns=["RegionId_y"], inplace=True)
merged_df.rename(columns={"RegionId_x": "RegionId"}, inplace=True)

#### Same goes here as well, I dropped Duplicate Column name with same Id and Renamed one RegionId_x with RegionId.

In [22]:
# Merging Region dataset
merged_df = merged_df.merge(
    region_df,
    on = "RegionId",
    how = "left"
)

print("After Merging the Region Dataset:", merged_df.shape)

After Merging the Region Dataset: (52930, 15)


### Merging Continent Data

In [ ]:
# Checking for ContinentId_y and Renaming ContinentId_x to ContinentId
merged_df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode',
       'AttractionId', 'Rating', 'ContinentId_x', 'RegionId', 'CountryId',
       'CityId', 'CityName', 'Country', 'Region', 'ContinentId_y'],
      dtype='str')

In [25]:
# Dropping ContinentId_y and Renaming ContinentId_x to ContinentId
merged_df.drop(columns=["ContinentId_y"], inplace=True)
merged_df.rename(columns={"ContinentId_x": "ContinentId"}, inplace=True)

In [26]:
# Merging Continent Dataset
merged_df = merged_df.merge(
    continent_df,
    on = "ContinentId",
    how = "left"
)

print("After Merging the Continent Dataset:", merged_df.shape)

After Merging the Continent Dataset: (52930, 15)


### Merging Item Dataset

In [27]:
merged_df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode',
       'AttractionId', 'Rating', 'ContinentId', 'RegionId', 'CountryId',
       'CityId', 'CityName', 'Country', 'Region', 'Continent'],
      dtype='str')

In [29]:
# Merging Item Dataset
merged_df = merged_df.merge(
    item_df,
    on = "AttractionId",
    how = "left"
)

print("After Merging the Item Dataset:", merged_df.shape)

After Merging the Item Dataset: (52930, 19)


### Merging Type Data

In [ ]:
# Checking merged_df columns names with AttributionType
merged_df.columns

Index(['TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode',
       'AttractionId', 'Rating', 'ContinentId', 'RegionId', 'CountryId',
       'CityId', 'CityName', 'Country', 'Region', 'Continent',
       'AttractionCityId', 'AttractionTypeId', 'Attraction',
       'AttractionAddress'],
      dtype='str')

In [31]:
# Merging Attraction Type
merged_df = merged_df.merge(
    type_df,
    on="AttractionTypeId",
    how="left"
)

print("After merging Attraction Type:", merged_df.shape)

After merging Attraction Type: (52930, 20)


In [ ]:
# Add Visit Mode
if "VisitModeId" in merged_df.columns:
    merged_df = merged_df.merge(
        mode_df,
        on="VisitModeId",
        how="left"
    )
    print("After merging Visit Mode:", merged_df.shape)
else:
    print("VisitMode already present in Transaction table")

VisitMode already present in Transaction table


In [34]:
# Checking the Merged data overview
display(merged_df.head())
merged_df.shape

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating,ContinentId,RegionId,CountryId,CityId,CityName,Country,Region,Continent,AttractionCityId,AttractionTypeId,Attraction,AttractionAddress,AttractionType
0,3,70456,2022,10,2,640,5,5,21,163,4341.0,Guildford,United Kingdom,Western Europe,Europe,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
1,8,7567,2022,10,4,640,5,2,8,48,464.0,Ontario,Canada,Northern America,America,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
2,9,79069,2022,10,3,640,5,2,9,54,774.0,Brazil,Brazil,South America,America,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
3,10,31019,2022,10,3,640,3,5,17,135,583.0,Zurich,Switzerland,Central Europe,Europe,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas
4,15,43611,2022,10,2,640,3,5,21,163,1396.0,Manchester,United Kingdom,Western Europe,Europe,1,63,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas


(52930, 20)

## Post-Merge Data Quality checks

### Overview of Merge Data

In [35]:
# Chacking the Shape and Columns of Merged Data
print("Merged Dataset Shape:", merged_df.shape)
print("\nColumns in Merged Dataset:")
display(merged_df.columns.tolist())

Merged Dataset Shape: (52930, 20)

Columns in Merged Dataset:


['TransactionId',
 'UserId',
 'VisitYear',
 'VisitMonth',
 'VisitMode',
 'AttractionId',
 'Rating',
 'ContinentId',
 'RegionId',
 'CountryId',
 'CityId',
 'CityName',
 'Country',
 'Region',
 'Continent',
 'AttractionCityId',
 'AttractionTypeId',
 'Attraction',
 'AttractionAddress',
 'AttractionType']

In [36]:
# Checking the Missing Values
missing_summary = merged_df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_summary / len(merged_df)) * 100

missing_df = pd.DataFrame({
    "Missing_Count": missing_summary,
    "Missing_Percentage": missing_percentage
})

display(missing_df)

,Missing_Count,Missing_Percentage
CityName,8,0.015114
CityId,8,0.015114
TransactionId,0,0.000000
UserId,0,0.000000
VisitMonth,0,0.000000
VisitYear,0,0.000000
VisitMode,0,0.000000
AttractionId,0,0.000000
ContinentId,0,0.000000
Rating,0,0.000000


#### Here, CityName and CityId has a minor null values, rest of the data is perfect till now, So i will fill those null values with the respective data types technique.

In [37]:
# Duplicate rows check
duplicate_rows = merged_df.duplicated().sum()
print("Total Duplicate Rows:", duplicate_rows)

Total Duplicate Rows: 0


In [38]:
# TransactionalId Uniqueness
print("TransactionId Unique:", merged_df["TransactionId"].is_unique)

TransactionId Unique: True


#### Suppose, the TransactionalId uniqueness was False I had to check and confirm where is the issue and solve it by removing the duplicate one's.

In [42]:
# Checking Rating Distribution
display(merged_df["Rating"].describe())

print("\nUnique Rating Values:")
#print(sorted(merged_df["Rating"].dropna().unique()))
print(sorted(int(x) for x in merged_df["Rating"].dropna().unique()))

count    52930.000000
mean         4.157699
std          0.970543
min          1.000000
25%          4.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: Rating, dtype: float64


Unique Rating Values:
[1, 2, 3, 4, 5]


In [43]:
# Visit Mode Check
display(merged_df["VisitMode"].value_counts(dropna=False))

VisitMode
2    21620
3    15217
4    10945
5     4525
1      623
Name: count, dtype: int64

In [44]:
# Orphan Records Check
print("Missing UserId info:", merged_df["UserId"].isnull().sum())
print("Missing AttractionId info:", merged_df["AttractionId"].isnull().sum())

Missing UserId info: 0
Missing AttractionId info: 0


## Data Cleaning Process

### First creating a New variable

In [45]:
clean_df = merged_df.copy()
print("Clean dataset created")
print("Shape:", clean_df.shape)

Clean dataset created
Shape: (52930, 20)


In [46]:
# Handling Missing Values in CityName and CityId

if "CitName" in clean_df.columns:
    clean_df['CityName'] = clean_df["CityName"].fillna("Unknown")

if "CityId" in clean_df.columns:
    clean_df["CityId"] = clean_df["CityId"].fillna(-1)

print("City Related missing values handled")

City Related missing values handled


In [47]:
# Re-checking the Missing values
clean_df.isnull().sum()

TransactionId        0
UserId               0
VisitYear            0
VisitMonth           0
VisitMode            0
AttractionId         0
Rating               0
ContinentId          0
RegionId             0
CountryId            0
CityId               0
CityName             8
Country              0
Region               0
Continent            0
AttractionCityId     0
AttractionTypeId     0
Attraction           0
AttractionAddress    0
AttractionType       0
dtype: int64

## Data types Check

In [48]:
# checking the cleaned dataset data types
clean_df.dtypes

TransactionId          int64
UserId                 int64
VisitYear              int64
VisitMonth             int64
VisitMode              int64
AttractionId           int64
Rating                 int64
ContinentId            int64
RegionId               int64
CountryId              int64
CityId               float64
CityName                 str
Country                  str
Region                   str
Continent                str
AttractionCityId       int64
AttractionTypeId       int64
Attraction               str
AttractionAddress        str
AttractionType           str
dtype: object

### Ensuring ID Columns are Numeric (int) Types

In [49]:
id_columns = [
    "TransactionId",
    "UserId",
    "CityId",
    "CountryId",
    "RegionId",
    "ContinentId",
    "AttractionId",
    "AttractionTypeId"
]

for col in id_columns:
    if col in clean_df.columns:
        clean_df[col] = clean_df[col].astype("int64", errors="ignore")

print("ID columns validated")

ID columns validated


### Fixing Numeric Columns for Future perpective

In [50]:
numeric_columns = [
    "VisitYear",
    "VisitMonth",
    "Rating"
]

for col in numeric_columns:
    if col in clean_df.columns:
        clean_df[col] = pd.to_numeric(clean_df[col], errors="coerce")

print("Numeric columns checked and fixed where necessary")

Numeric columns checked and fixed where necessary


In [52]:
# Re-checking the data types
clean_df.dtypes

TransactionId        int64
UserId               int64
VisitYear            int64
VisitMonth           int64
VisitMode            int64
AttractionId         int64
Rating               int64
ContinentId          int64
RegionId             int64
CountryId            int64
CityId               int64
CityName               str
Country                str
Region                 str
Continent              str
AttractionCityId     int64
AttractionTypeId     int64
Attraction             str
AttractionAddress      str
AttractionType         str
dtype: object